<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/machine_translation/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [ ]:
!pip install tokenizers matplotlib sklearn

In [ ]:
!pip install tokenizers 

In [ ]:
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en

In [ ]:
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [ ]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

In [ ]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [ ]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [ ]:
#tokenizer_en.save('tokenizer_en')
#tokenizer_ru.save('tokenizer_ru')

In [ ]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [ ]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids 

In [ ]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, True) for t in ru_sents]

In [ ]:
max_len_en = np.max([len(x) for x in X_en])
max_len_ru = np.max([len(x) for x in X_ru])

In [ ]:
max_len_en, max_len_ru = 45, 48

In [ ]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')

In [ ]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_ru_out = tf.keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post', 
              value=PAD_IDX)

X_ru_dec = tf.keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1, 
              padding='post', value=PAD_IDX)

In [ ]:
(X_en_train, X_en_valid, 
X_ru_dec_train, X_ru_dec_valid, 
X_ru_out_train, X_ru_out_valid) = train_test_split(X_en, 
                                                  X_ru_dec, 
                                                  X_ru_out, 
                                                  test_size=0.05)

Transformer

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
    # Считаем скалярное произведение между запросом (query) и ключом (key), транспонируя ключ
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # Получаем глубину (размерность) ключа и преобразуем ее во float
    depth = tf.cast(tf.shape(key)[-1], tf.float32)

    # Делим результат скалярного произведения на квадратный корень из глубины
    # Это делается для уменьшения влияния больших значений и стабилизации градиентов во время обучения
    logits = matmul_qk / tf.math.sqrt(depth)

    # Если есть маска, применяем ее к логитам, чтобы обнулить нежелательные значения
    if mask is not None:
        logits += (mask * -1e9)

    # Применяем функцию softmax для получения весов внимания
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # Умножаем веса внимания на значения (value) для получения итогового результата
    output = tf.matmul(attention_weights, value)

    return output

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads  # количество голов для внимания
        self.d_model = d_model  # размерность вектора модели

        # Убеждаемся, что размерность модели делится нацело на количество голов
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads  # размерность каждой головы

        # Создаем полносвязные слои для запроса, ключа и значения
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        # Создаем последний полносвязный слой
        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        # Разделяем входные данные на головы
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Пропускаем запрос, ключ и значение через соответствующие полносвязные слои
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Разделяем запрос, ключ и значение на головы 
        # то есть просто разрезаем вектора на num_heads частей 
        # и сравниваем все части между собой
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Выполняем механизм внимания с масштабированным скалярным произведением
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # Объединяем головы вместе (склеиваем векторы в один)
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # Пропускаем объединенное внимание через дополнительный полносвязный слой
        # Он просто добавляет сложности нашей модели
        outputs = self.dense(concat_attention)

        return outputs

In [ ]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    #call_mha
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    #inputs (они тут называются outputs но это просто такой нейминг, 
    # этот параметр передается в encoder_layer первым и encoder_layer будет считать его inputs)
    # outputs он тут называется для удобства, так как он будет перезаписываться
    # чтобы на вход следующему блоку подавать уже не эмбединги,
    # а то что получится как результат предыдущего блока
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                          vocab_size=vocab_size[0],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[0],
                        )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                          vocab_size=vocab_size[1],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[1],
                        )(inputs=[dec_inputs, enc_outputs, 
                                  look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [ ]:
tf.keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8

UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


model = transformer(
    vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    max_len=[max_len_en, max_len_ru])


optimizer = tf.keras.optimizers.Adam(
    0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                            monitor='val_loss',
                                            verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

In [ ]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train, 
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=500,
             epochs=1,
             callbacks=[checkpoint]
             )

1900/1900 [==============================] - ETA: 0s - loss: 1.3720 - accuracy: 0.1090
Epoch 1: val_loss improved from inf to 0.97220, saving model to model_ruen
1900/1900 [==============================] - 1602s 833ms/step - loss: 1.3720 - accuracy: 0.1090 - val_loss: 0.9722 - val_accuracy: 0.1464


In [ ]:
def translate(text):
    input_ids = encode(text, tokenizer_en, target=False)

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post', 
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX)

    
    
    output_ids = [tokenizer_ru.token_to_id('[START]') ]
    
    pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False).numpy()
 
    
    while pred.argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[END]')]:
        if len(output_ids) >= max_len_ru:
            break
        # можно занизить скор тэга UNK чтобы он никогда не генерировался
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100

        output_ids.append(pred.argmax(2)[0][-1])
        pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False).numpy()

    return tokenizer_ru.decode(output_ids[1:], )

In [ ]:
%pip install nltk

In [ ]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


In [ ]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

In [ ]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [ ]:
translations = []

for i in range(len(en_sents_test)):
    translations.append(translate(en_sents_test[i]))

In [ ]:
bleus = []

for i, t in enumerate(translations):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True))

In [ ]:
(sum(bleus)/len(bleus))*100

8.49825966067874


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

**Backtranslation** - один из методов, относящихся к технике Data augmentation (увеличения данных), которая применяется при недостатке обучающих данных (часто при работе с малоресурсными языками). 
Идея обратного перевода опирается на предположение, что существующие небольшие параллельные корпуса можно дополнить синтетическими битекстами, основанными на материалах одноязычного корпуса. 
Предположим, у нас есть небольшой параллельный корпус en-ru, где en - исходный язык, ru - целевой, а также много данных на ru. При обратном переводе модель сначала обучается на существующем параллельном корпусе в обратную сторону - перевод «ЦЯ-ИЯ». Эту модель используется для перевода одноязычных данных ru на en. Таким образом мы получаем синтетический параллельный корпус, данные которого можем перемешать с изначальным параллельным корпусом и затем обучить модель на всех имеющихся данных уже в направлении «ИЯ-ЦЯ».